NLP with Stateful RNNs

We will try to predict Shakepeare Text with an RNN.

Contrary to a stateless RNN, a stateful RNN does not throw away the hidden states at every time step and this leads to a model better suited at learning long term patterns.

In [ ]:
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
pip install tensorflow==2.0.0-beta1

In [ ]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

We download Shakespeare text dataset using keras get_file.

In [ ]:
"".join(sorted(set(shakespeare_text.lower())))

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

We use kera's tokenizer class to encode every character as an integer.

It will find all the characters used in the text and map them to a unique character ID.

In [ ]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100

Let's also split our data into a training set, validation set, and test set.

In this case we need to avoid any overlap between each set.

We take 90% of the data for the training set in this case.

In [ ]:
n_steps = 100
window_length = n_steps

We use the dataset's window method to change this long sequence of characters into multiple smaller windows of text.

Smaller n_steps makes it easier to train with smaller inputs but limits the length of pattern the RNN can learn.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.repeat().batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

Since the windows method uses a nested dataset which cannot be used for training, we must convert it to a tensor using the flat_map to convert it into a flat dataset of tensors.

Since each window is of the same size, we use the batch(window_length) on each window to get a single tensor on each of them.

In [ ]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

After shuffling the windows, we batch them and seperate the inputs from the last character.

Since categorical input features should generally be encoded, we encode each character using one-hot vector.

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

We are training a model to predict the next character using the previous 100 characters.

We use 2 GRU layers with 128 units and 20% dropout on the dataset and the hidden states.

The output layer is simply a dense layer with 39 units since we find 39 unique characters in the text.

We compile this model with spare_categorical_crossentropy loos and Adam optimizer.

We use stateful=True since we want a statefull RNN.

We also need a batch_input_shape in the first layer since the RNN needs to know the batch size.

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

We need to reset states after each epoch before we go to the start of the text using this callback.

In [ ]:
def preprocess(texts): 
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1 
    return tf.one_hot(X, max_id)

We need to preprocess the text before feeding it to the model.

In [ ]:
def next_char(text, temperature=1): 
    X_new = preprocess([text]) 
    y_proba = model.predict(X_new)[0, -1:, :] 
    rescaled_logits = tf.math.log(y_proba) / temperature 
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1 
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text, n_chars=50, temperature=1): 
    for _ in range(n_chars): 
        text += next_char(text, temperature) 
    return text

We want the next character being guessed to be random. Otherwise the model will predict the same word again and again.

A temparature close to 0 will favor high probability characters while a high temperature will give equal probability to all the characters.

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()
        

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps
model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=50,
                   callbacks=[ResetStatesCallback()])

We compile the model and this can take quite some time without a GPU.

In [ ]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

stateless_model.build(tf.TensorShape([None, None, max_id]))

stateless_model.set_weights(model.get_weights())
model = stateless_model

In [ ]:
print(complete_text("t"))

Our output: from most breathe life unto him, with care.

We finally have some Shakespeare text!

It could be better but still quite good.